In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [2]:
df = pd.read_csv('all_samples.csv')
df.head()

,Unnamed: 0,name,ILMN_1651217,ILMN_1651229,ILMN_1651234,ILMN_1651236,ILMN_1651237,ILMN_1651254,ILMN_1651259,ILMN_1651260,...,ILMN_1815885,ILMN_1815908,ILMN_1815923,ILMN_1815924,ILMN_1815933,ILMN_1815937,ILMN_1815938,ILMN_1815941,ILMN_1815951,CELIAC
0,0,0,4.229567,4.802085,4.145582,4.274502,4.268115,6.853804,4.401350,4.123169,...,4.376735,4.395501,4.338936,5.198647,4.594269,4.264604,4.256310,4.821757,5.005588,1
1,1,1,4.197183,4.820311,4.171221,4.332524,4.186809,6.663657,4.559615,4.278860,...,4.732124,4.417266,4.656831,4.615440,4.594269,4.336589,4.317376,4.518347,4.308311,1
2,2,2,4.131493,4.640774,4.075849,4.233316,4.334549,6.694727,4.370504,4.169419,...,4.292552,4.379864,4.211071,5.530672,4.570808,4.379545,4.241886,4.680351,4.780989,1
3,3,3,4.207410,4.508425,4.100585,4.166837,4.530517,6.506971,4.483179,4.242860,...,4.371180,4.406084,4.186757,5.358646,4.632107,4.282658,4.237614,4.602680,4.637598,1
4,4,4,4.245230,4.538779,4.040637,4.266853,4.326313,6.774611,4.409940,4.228860,...,4.345227,4.488653,4.364008,5.605900,4.624200,4.275774,4.251683,4.686359,4.687048,1


In [3]:
df.drop(columns=['Unnamed: 0', 'name'], inplace=True)
df.head()

,ILMN_1651217,ILMN_1651229,ILMN_1651234,ILMN_1651236,ILMN_1651237,ILMN_1651254,ILMN_1651259,ILMN_1651260,ILMN_1651261,ILMN_1651262,...,ILMN_1815885,ILMN_1815908,ILMN_1815923,ILMN_1815924,ILMN_1815933,ILMN_1815937,ILMN_1815938,ILMN_1815941,ILMN_1815951,CELIAC
0,4.229567,4.802085,4.145582,4.274502,4.268115,6.853804,4.401350,4.123169,4.639975,7.136778,...,4.376735,4.395501,4.338936,5.198647,4.594269,4.264604,4.256310,4.821757,5.005588,1
1,4.197183,4.820311,4.171221,4.332524,4.186809,6.663657,4.559615,4.278860,4.994493,6.803521,...,4.732124,4.417266,4.656831,4.615440,4.594269,4.336589,4.317376,4.518347,4.308311,1
2,4.131493,4.640774,4.075849,4.233316,4.334549,6.694727,4.370504,4.169419,5.093272,6.720391,...,4.292552,4.379864,4.211071,5.530672,4.570808,4.379545,4.241886,4.680351,4.780989,1
3,4.207410,4.508425,4.100585,4.166837,4.530517,6.506971,4.483179,4.242860,5.138309,6.881151,...,4.371180,4.406084,4.186757,5.358646,4.632107,4.282658,4.237614,4.602680,4.637598,1
4,4.245230,4.538779,4.040637,4.266853,4.326313,6.774611,4.409940,4.228860,4.948306,6.847382,...,4.345227,4.488653,4.364008,5.605900,4.624200,4.275774,4.251683,4.686359,4.687048,1


In [ ]:
target = df['CELIAC']

In [ ]:
# from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold # maintains class proportions

In [ ]:
skf = StratifiedKFold()
skf.split(target)

In [ ]:
# Support vector machines (recommended for high-dimensional data; can work with small sample size)
from sklearn.svm import SVC
from sklearn.feature_selection import SelectPercentile 
from sklearn.pipeline import Pipeline

In [ ]:
svm = Pipeline([
    ('feature_selection', SelectPercentile(percentile=20)),
    ('support_vector_machine', SVC(verbose=True))
])

In [4]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import classification_report